In [1]:
import ollama
import pandas as pd
import chromadb

In [2]:
def setup_chromadb():
    # read Gekko LLM training data
    url='https://raw.githubusercontent.com'
    path='/BYU-PRISM/GEKKO/master/docs/llm/train.jsonl'
    qa = pd.read_json(url+path,lines=True)
    documents = []
    metadatas = []
    ids = []

    for i in range(len(qa)):
        s = f"### Question: {qa['question'].iloc[i]} ### Answer: {qa['answer'].iloc[i]}"
        documents.append(s)
        metadatas.append({'qid': f'qid_{i}'})
        ids.append(str(i))

    cc = chromadb.Client()
    cdb = cc.create_collection(name='gekko')
    cdb.add(documents=documents, metadatas=metadatas, ids=ids)
    return cdb

# Setup ChromaDB
cdb = setup_chromadb()

C:\Users\DAY\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:14<00:00, 5.68MiB/s]  


In [3]:
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

In [4]:
# Define the RAG chain
def rag_chain(question, cdb):
    context = cdb.query(query_texts=[question],
                        n_results=5, include=['documents'])
    formatted_context = "\n\n".join(x for x in context['documents'][0])
    formatted_context += "\n\nYou are a professional and technical assistant trained to answer questions about Gekko, which is a high-performance Python package for optimization, simulation, machine learning, data-science, model predictive control, and parameter estimation. In addition, you can also help with answering questions about programming in Python, particularly in relation to the aforementioned topics. Your primary goal is to assist users in finding solutions and gaining knowledge in these areas."
    result = ollama_llm(question, formatted_context)
    return result

In [5]:
question = 'What are you trained to do?'
out = rag_chain(question, cdb)
print(out)

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [6]:
from gekko import support
assistant = support.agent()
assistant.ask("What are you trained to do?")

 I am trained to answer questions about Gekko, which is a high-performance Python package for optimization, simulation, machine learning, data-science, model predictive control, and parameter estimation. I can also help with answering questions about programming in Python, particularly in relation to these topics. My main goal is to assist users in finding solutions and gaining knowledge in these areas. My responses are based on a Large-Language Model (LLM) that has been fine-tuned on hundreds of Gekko examples and typical user questions. If you have questions about cybersecurity applications, I can help by assessing and mitigating risks from computer-based adversarial attacks on engineered systems, and suggest using Gekko for modeling or regression to detect bad actors. For further consulting support, users are referred to Advanced Process Solutions (APS) services and software at apopt.com. Additional resources such as documentation and a user community are available from Gekko ReadTheDocs and StackOverflow with tag gekko.